# Extractive HuggingFace

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/qa-extractive-huggingface](https://github.com/huseinzol05/Malaya/tree/master/example/qa-extractive-huggingface).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
%%time

import malaya
from pprint import pprint

CPU times: user 3.13 s, sys: 3.33 s, total: 6.46 s
Wall time: 2.28 s


/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3890
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### What is Extractive QA

is a reading comprehension problem, eg,

```json
{
    'title': 'Normans',
    'paragraphs': [
        {
            'context': 'Orang Norman (Norman: Nourmands; Perancis: Normands; Latin: Normanni) ialah orang-orang yang pada abad ke-10 dan ke-11 memberikan nama mereka kepada Normandy, sebuah wilayah di Perancis. Mereka diturunkan daripada Norse ("Norman" berasal daripada penyerang "Norseman") dan lanun dari Denmark, Iceland dan Norway yang, di bawah pimpinan mereka Rollo, bersetuju untuk bersumpah fealty kepada Raja Charles III dari Francia Barat. Melalui generasi asimilasi dan percampuran dengan penduduk asli Frankish dan Roman-Gaulish, keturunan mereka akan beransur-ansur bergabung dengan budaya Carolingian yang berpusat di Francia Barat. Identiti budaya dan etnik yang berbeza dari orang Norman muncul pada mulanya pada separuh pertama abad ke-10, dan ia terus berkembang pada abad-abad yang berjaya.',
            'qas': [
                {
                    'question': 'Di negara manakah Normandy berada?',
                    'answers': [
                        {'text': 'Perancis', 'answer_start': 177},
                        {'text': 'Perancis', 'answer_start': 177},
                        {'text': 'Perancis', 'answer_start': 177},
                        {'text': 'Perancis', 'answer_start': 177},
                    ],
                    'id': '56ddde6b9a695914005b9628',
                    'is_impossible': False,
                }
            ],
        }
    ],
}
```

So we need to give a long paragraph and multiple questions, and the model will return answers based on that paragraph with start and end spans. Read more about SQUAD dataset https://rajpurkar.github.io/SQuAD-explorer/

### List available HuggingFace models

In [4]:
malaya.qa.extractive.available_huggingface()

INFO:malaya.qa.extractive:tested on translated SQUAD V2 Dev set, https://github.com/huseinzol05/malay-dataset/tree/master/question-answer/squad
INFO:malaya.qa.extractive:tested on SQUAD V2 Dev set, https://rajpurkar.github.io/SQuAD-explorer/


,Size (MB),exact-ms,f1-ms,exact-en,f1-en,Suggested length
mesolitica/finetune-extractive-qa-t5-tiny-standard-bahasa-cased,139.0,61.97503,61.97503,61.97503,61.97503,256.0
mesolitica/finetune-extractive-qa-t5-small-standard-bahasa-cased,242.0,61.97503,61.97503,61.97503,61.97503,256.0
mesolitica/finetune-extractive-qa-t5-base-standard-bahasa-cased,892.0,61.97503,61.97503,61.97503,61.97503,256.0


### Load HuggingFace model

In [5]:
model = malaya.qa.extractive.huggingface(model = 'mesolitica/finetune-extractive-qa-t5-base-standard-bahasa-cased')

### Copy from wikipedia and news

In [6]:
# https://ms.wikipedia.org/wiki/Mohd_Najib_bin_Abdul_Razak

p_wikipedia = """
Najib razak telah dipilih untuk Parlimen Malaysia pada tahun 1976, 
pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang. 
Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang, 
sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan. 
Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran. 
Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi, 
sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008. 
Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013, 
walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""
q_wikipedia = ['pada umur berapa najib razak menggantikan ayahnya', 
               'Apakah jawatan yang pernah dipegang oleh Najib Razak']

In [7]:
# https://www.malaysiakini.com/news/574914

p_news = """
Bekas perdana menteri Najib Razak mempersoalkan tindakan polis yang menurutnya tidak serta-merta mengeluarkan kenyataan berhubung dakwaan Adun Perikatan Nasional (PN) "merancang" insiden rogol.
Sedangkan, kata ahli parlimen Pekan itu, polis pantas mengeluarkan kenyataan apabila dia dilapor terlupa mengimbas MySejahtera sebelum masuk restoran.
"Berita Najib lupa scan MySejahtera tular, kenyataan polis terus keluar. Berita Dr Mahathir Mohamad lupa scan, kenyataan, polis serta-merta keluar.
"Sebab itu saya pelik kenapa pihak polis belum sempat keluar apa-apa kenyataan berhubung kes seorang gadis membuat laporan polis untuk dakwa Adun PN rancang insiden rogolnya," katanya di Facebook hari ini.
Najib merujuk dakwaan seorang wanita yang mendakwa dirogol kenalan kepada Adun Gombak Setia, Hilman Idham.
Wanita itu mendakwa ahli politik dari Bersatu berkenaan merancang insiden yang berlaku pada 5 Dis lalu.
Menurut laporan polis pada 8 Mei, mangsa mendakwa kejadian itu berlaku di sebuah hotel di Selangor, yang pada masa itu berada di bawah perintah kawalan pergerakan bersyarat (PKPB).
"""

q_news = ['siapakah yang mempersoalkan tindakan polis', 'siapakah Adun Gombak Setia']

### Predict

```python
def predict(
    self,
    paragraph_text: str,
    question_texts: List[str],
    **kwargs,
):
    """
    Predict Span from questions given a paragraph.

    Parameters
    ----------
    paragraph_text: str
    question_texts: List[str]
        List of questions, results really depends on case sensitive questions.
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

    Returns
    -------
    result: List[str]
    """
```

In [8]:
model.predict(p_wikipedia, q_wikipedia)

['23', 'Menteri Pertahanan dan Menteri Pelajaran']

In [9]:
model.predict(p_news, q_news)

['bekas perdana menteri Najib Razak', 'Hilman Idham']

### Able to mixed MS and EN

In [10]:
# https://ms.wikipedia.org/wiki/Mohd_Najib_bin_Abdul_Razak

p_wikipedia = """
Najib razak telah dipilih untuk Parlimen Malaysia pada tahun 1976, 
pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang. 
Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang, 
sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan. 
Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran. 
Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi, 
sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008. 
Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013, 
walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""
q_wikipedia = ['pada umur berapa najib razak menggantikan ayahnya', 
               'Apakah jawatan yang pernah dipegang oleh Najib Razak',
              'what age najib razak replaced his father']

In [11]:
model.predict(p_wikipedia, q_wikipedia)

['23', 'Menteri Pertahanan dan Menteri Pelajaran', '23']

In [14]:
# https://www.malaysiakini.com/news/574914

p_news = """
Bekas perdana menteri Najib Razak mempersoalkan tindakan polis yang menurutnya tidak serta-merta mengeluarkan kenyataan berhubung dakwaan Adun Perikatan Nasional (PN) "merancang" insiden rogol.
Sedangkan, kata ahli parlimen Pekan itu, polis pantas mengeluarkan kenyataan apabila dia dilapor terlupa mengimbas MySejahtera sebelum masuk restoran.
"Berita Najib lupa scan MySejahtera tular, kenyataan polis terus keluar. Berita Dr Mahathir Mohamad lupa scan, kenyataan, polis serta-merta keluar.
"Sebab itu saya pelik kenapa pihak polis belum sempat keluar apa-apa kenyataan berhubung kes seorang gadis membuat laporan polis untuk dakwa Adun PN rancang insiden rogolnya," katanya di Facebook hari ini.
Najib merujuk dakwaan seorang wanita yang mendakwa dirogol kenalan kepada Adun Gombak Setia, Hilman Idham.
Wanita itu mendakwa ahli politik dari Bersatu berkenaan merancang insiden yang berlaku pada 5 Dis lalu.
Menurut laporan polis pada 8 Mei, mangsa mendakwa kejadian itu berlaku di sebuah hotel di Selangor, yang pada masa itu berada di bawah perintah kawalan pergerakan bersyarat (PKPB).
"""

q_news = ['siapakah yang mempersoalkan tindakan polis', 'siapakah Adun Gombak Setia',
         'who is the victim on 8 may incident',
         'where does the 8 may incident happened']

In [15]:
model.predict(p_news, q_news)

['bekas perdana menteri Najib Razak',
 'Hilman Idham',
 'wanita',
 'sebuah hotel di Selangor']